## Scrapping alumni info from pre-scrapped and downloaded links
1. Automation libraries:
<br> *selenium* -- controls web browsers through programs and performs browser automation.
2. Scrapping library:
<br> *bs4* --pulls data out of HTML files
3. Anti-scrapping prevention library:
<br> *time* -- suspends execution for the given number of seconds

In [1]:
import time, random
from bs4 import BeautifulSoup
from selenium import webdriver
import glob
import os 
import pandas as pd

In [1]:
## open browser and login LinkedIn account 
browser = webdriver.Chrome('you local address/chromedriver')
browser.get('http://www.linkedin.com/uas/login')

NameError: name 'webdriver' is not defined

In [3]:
## open file and get individual profile link
profile_link = []
for filepath in glob.glob(os.path.join('the path of the downloaded individual profile link', '*.html')):
    with open(filepath) as fp:
        soup_test = BeautifulSoup(fp, 'html.parser')
        ## get profile link of the whole page 
        soup_test.find_all('dt', class_='result-lockup__name')
        for links in soup_test.find_all('dt', class_='result-lockup__name'):
            alst = links.find_all('a')
            profile_link.extend([e['href'] for e in alst])
       

In [22]:
search_school_name = "Brown University"

In [ ]:
lst_record = []
for link in profile_link[:]:
    result = []
    individual_record = {}
    time.sleep(random.uniform(1,2))
    browser.get(link)
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    top_card = soup.find('div', {'class': 'profile-topcard-content-container mr2'})
    try:
        text = top_card.get_text()
        text = text.split('\n')
    except AttributeError:
        pass 
    for t in text:
        if t.strip():
            result.append(t.strip())
    for i in range(len(result)):
        if "connections" in result[i]:
            try:
                individual_record["current_location"] = result[i-1]
            except IndexError:
                pass
        if "Current" in result[i]:
            try:
                individual_record["current_work"] = result[i+1]
            except IndexError:
                pass
        if "Previous" in result[i]:
            try:
                individual_record["previous_work"] = result[i+1]
            except IndexError:
                pass
        if "Education" in result[i]:
            try:
                individual_record["latest_Education"] = result[i+1]
            except IndexError:
                pass 
        if "NAME_SEARCH" in link:
            degree = soup.find('span',{'data-test-education':"field"})
            try:
                degree = degree.getText().strip()
                individual_record["major"] = degree
            except AttributeError:
                 pass 
    if individual_record not in lst_record:
        lst_record.append(individual_record)
        

In [9]:
## write out csv file 
df = pd.DataFrame(lst_record)
df['search_school'] = search_school_name
filename = str(search_school_name)+".csv"
df.to_csv(filename,index = True)